In [6]:
import os
# Change the working directory
os.chdir('/private/home/yxu267/anaconda3/envs/dimelo/lib/python3.10/site-packages')

# Verify the change
print(os.getcwd())



/private/home/yxu267/anaconda3/envs/dimelo/lib/python3.10/site-packages


In [5]:
import pysam
import numpy as np
from Bio import SeqIO
import time 
import matplotlib.patches as patches
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import itertools
import pandas as pd 
from tabulate import tabulate
import csv
import random

In [18]:
import pysam
import numpy as np
from Bio import SeqIO
import time 
import matplotlib.patches as patches
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import itertools
import pandas as pd 
from tabulate import tabulate
import csv
import random

np.set_printoptions(threshold=np.inf)
min_quality_score = 8

#Load the bam file 
bamfile = pysam.AlignmentFile(
    "/private/groups/migalab/dan/data_analysis/mC_border_analysis/CENPA_dimelo_AS_whole_merge_filtered.bam",
    "rb") 


assembly_ = open("/private/groups/migalab/dan/reference/hg002v1.0.1.fasta", "r")

start_time = time.time()

#Load the reference genome and make it into a dictionary 
fasta_sequences = SeqIO.parse(assembly_, "fasta")
assembly={}
for fasta in fasta_sequences:
    name, sequence = fasta.id, str(fasta.seq)
    assembly[name] = sequence

#Make a dictionary for all the chromosomes and their corresponding sequence length 
assembly_sequence_length = {}    
for chromosome in assembly:
    assembly_sequence_length[chromosome] = len(assembly[chromosome])
    
end_time = time.time()
elapsed_time = end_time - start_time
print (elapsed_time, "seconds")
assembly_.close()



[W::hts_idx_load3] The index file is older than the data file: /private/groups/migalab/dan/data_analysis/mC_border_analysis/CENPA_dimelo_AS_whole_merge_filtered.bam.csi


58.8197660446167 seconds


In [19]:
'''The purpose of the code here is to make an active array where the chromosome name is
the dictionary key. The active array blocks are in lists inside the key.
'''
import os
input_file = '/private/groups/migalab/dan/data_analysis/alpha_bed/hg002v1.0.fasta.manualAlpha.cenSat_H1L_merged.bed'
active_dict = {}
with open(input_file, 'r') as infile:  
    num = 0 
    previous_chr_num = ""
    previous_parental_status = ""
    for i in infile:
        chr_num = i.split ('_')[0]
        parental_status = i.split ('_')[1][0:8]
        if chr_num == previous_chr_num and parental_status !=previous_parental_status:
            num = 0 
        elif chr_num == previous_chr_num and parental_status ==previous_parental_status:
            pass 
        else: 
            num = 0 
        active = i.split ()
        if active[0] not in active_dict:
            
            active_dict[active[0]] = [[int(active[1]) ,int(active[2])]]
        else:
            active_dict[active[0]].append([int(active[1]) ,int(active[2])])
        previous_chr_num = chr_num 
        previous_parental_status = parental_status


In [21]:
'''The purpose of the code here is to make an active array where the chromosome name is
the dictionary key. The active array blocks are in lists inside the key.
'''
import os
input_file = '/private/groups/migalab/dan/data_analysis/alpha_bed/hg002v1.0.fasta.manualAlpha.cenSat_H1L_merged.bed'
active_dict = {}
with open(input_file, 'r') as infile:  
    num = 0 
    previous_chr_num = ""
    previous_parental_status = ""
    for i in infile:
        chr_num = i.split ('_')[0]
        parental_status = i.split ('_')[1][0:8]
        if chr_num == previous_chr_num and parental_status !=previous_parental_status:
            num = 0 
        elif chr_num == previous_chr_num and parental_status ==previous_parental_status:
            pass 
        else: 
            num = 0 
        active = i.split ()
        if active[0] not in active_dict:
            
            active_dict[active[0]] = [[int(active[1]) ,int(active[2])]]
        else:
            active_dict[active[0]].append([int(active[1]) ,int(active[2])])
        previous_chr_num = chr_num 
        previous_parental_status = parental_status


In [20]:
active_array_range={}
for chromosome in active_dict:
    active_array_range[chromosome] = []
    for fragment in range (active_dict[chromosome][0][0], active_dict[chromosome][0][1],1000):

        active_array_range[chromosome].append ([fragment,fragment+1000])
        

In [28]:
''' here in the code, I am formulating the CDR regions and listing the CDRs in each and every chromosome'''
input_file = '/private/groups/migalab/dan/data_analysis/CDRStrictBedFiles/HG002v1.0_Guppy_merged.winnowmap.sort.secphase.viterbi_CDR.bed'
CDR_dict = {}
with open(input_file, 'r') as infile:  
    for i in infile:
        chr_num = i.split('\t')[0]
        CDR_start = int (i.split('\t')[1]) - 1000
        CDR_end = int(i.split('\t')[2].split('\n')[0]) + 1000
        if chr_num not in CDR_dict:
            CDR_dict[chr_num] = [[CDR_start ,CDR_end]]
        elif chr_num in CDR_dict:  
            CDR_dict[chr_num].append ([CDR_start ,CDR_end])

'''This piece of code is to determine how many CDRs there are in a given chromosome'''
for i in CDR_dict:
    for CDR in CDR_dict[i]:
        CDR_length = int(CDR[1]) - int(CDR[0])
        total_A_count = assembly[i][int(CDR[0]):int(CDR[1])].count ("A")



In [26]:
'''Based on the CDR regions, I am obtaining CDR adjacent regions in the same format as the CDR data set above'''
CDR_adjacent = {}
for chromosome in CDR_dict: 
    CDR_adjacent[chromosome] =[]
    for CDR in CDR_dict[chromosome]: 
        #print (CDR) 
        CDR_adjacent_left_space = [int(CDR[0]) - 1001, int(CDR[0]) - 1]
        CDR_adjacent_right_space = [int(CDR[1]) + 1, int(CDR[1]) + 1001]
        
        CDR_adjacent[chromosome].append (CDR_adjacent_left_space)
        CDR_adjacent[chromosome].append (CDR_adjacent_right_space)
        
    

In [27]:
none_CDR_active = {}
for chromosome in CDR_dict: 
    none_CDR_active[chromosome] = [] 
    active_start = active_dict[chromosome][0][0]
    active_end = active_dict[chromosome][0][1]
    CDR_start_position = int (CDR_dict[chromosome][0][0]) - 1 
    CDR_end_position = int (CDR_dict[chromosome][-1][1]) + 1
    none_CDR_active[chromosome].append ([active_start, CDR_start_position])
    for coordinate in range(len(CDR_dict[chromosome]) - 1):
        # Generate in-between coordinates
        
        start_of_next = int (CDR_dict[chromosome][coordinate + 1][0]) - 1 
        end_of_current = int (CDR_dict[chromosome][coordinate][1]) + 1 
        # Ensure there's no overlap and the next start is greater than the current end
        if int(start_of_next) > int(end_of_current):
            none_CDR_active[chromosome].append([end_of_current, start_of_next])
    
    none_CDR_active[chromosome].append ([CDR_end_position, active_end])


In [40]:
def chromosome_arm_random_region (segment_num, chr_name, H1L_active_dict, assembly_sequence_length): 
    # create variables to contain regions chosen 
    chromosome_arm_regions = []
    excluded_portion = []
    H1L_active_length = 0 
    
    for active_region in H1L_active_dict[chr_name]: 
        #calculate the length of the active array region for each chromosome 
        H1L_active_length = active_region[1] - active_region[0]
        
        #defining where the chromosome arm regions are
        
        #The left side of the active array 
        chromosome_arm_regions.append([0,active_region[0] - 1])
        
        #The right side of the active array 
        chromosome_arm_regions.append([active_region[1] + 1,assembly_sequence_length[chr_name]])
        
        #calculate the percentage portion of where the active array is in and add them to the exclusion bin 
        start_portion = active_region[0] / H1L_active_length
        end_portion = active_region[1] / H1L_active_length
        excluded_portion.append (int(start_portion))
        excluded_portion.append (int(end_portion))
        
    #calculate the total portions of the active array 
    total_segment_amount = int(assembly_sequence_length[chr_name] / H1L_active_length)
    
    
    #pick defined amount of random numbers between 0 and the pre defined amount of random numbers 
    random_numbers = []
    for num in range(segment_num):
        while True: 
            #if the same random number gets picked twice, repeat 
            current_random_number = random.randint(0, total_segment_amount)
            if current_random_number not in (excluded_portion and random_numbers):
                break
            
            
        random_numbers.append(current_random_number)
    
    #Expand the chromosome portion number to chromosome position number by multiplying 
    random_picked_regions = [num * H1L_active_length for num in random_numbers]
    uncoded_region_list = []
    
    #Make a dictionary that contains randomly picked region for each chromosome 
    for item in random_picked_regions: 
        arms_region_start = item
        arms_region_end = item + H1L_active_length
        uncoded_region_list.append([arms_region_start, arms_region_end])
        
    return uncoded_region_list


chromosome_arm_random_region_dict = {}
for chromosome in CDR_dict: 
    chromosome_arm_random_region_dict[chromosome] = chromosome_arm_random_region (3,
                                                                             chromosome, 
                                                                             active_dict, 
                                                                             assembly_sequence_length)


In [41]:
def check_lists(cdr_adjacent):
    filtered_cdr_adjacent = {}
    
    for chromosome, regions in cdr_adjacent.items():
        valid_regions = []
        for region in regions:
            if region[1] > region[0]:
                valid_regions.append(region)
            else:
                print(f"Removed invalid region in {chromosome}: {region[1]} is not larger than {region[0]}")
        if valid_regions:
            filtered_cdr_adjacent[chromosome] = valid_regions

    return filtered_cdr_adjacent
CDR_dict= check_lists(CDR_dict)
CDR_adjacent = check_lists(CDR_adjacent)
none_CDR_active = check_lists(none_CDR_active)
chromosome_arm_random_region_dict = check_lists(chromosome_arm_random_region_dict)

In [37]:
'''
The idea of this function is to isolate the the desired regions (here in the function, it is called the subset) in the mod 
numpy array without dashes(insertions)'''

def mod_subset_producing_step (mod_no_dash,alignment_dash,target_start_no_dash,target_end_no_dash):
    #mod_no_dash = is the numpy array of the mod without any insertions and deletions
    # alignment_dash = is the alignment sequence with the dashes in it 
    # target_start = it's the subset starting position WITHOUT the dashes!!! 


    # Create a mask to identify non-dash positions
    mask = [char != '-' for char in alignment_dash]

    # Generate cumulative counts only for True values in the mask
    cumulative_counts = list(itertools.accumulate(mask))


    
    # Create the final indexes list
    indexes = [count - 1 if is_non_dash else '-' for count, is_non_dash in zip(cumulative_counts, mask)]



    target_start_dash = indexes.index (target_start_no_dash)

        
    try:
        target_end_dash = indexes.index (target_end_no_dash)
    except ValueError: 
        target_end_dash = indexes[-1]




    #obtain dashed alignment 
    alignment_dash_sequence_pre_subset = alignment_dash[0:target_start_dash]
    alignment_dash_sequence_subset = alignment_dash[target_start_dash:target_end_dash]

    #create no dash alignment 
    alignment_no_dash_sequence_pre_subset = alignment_dash_sequence_pre_subset.replace("-","")
    alignment_no_dash_sequence_subset = alignment_dash_sequence_subset.replace("-","")

    subset_no_dash_start = len(alignment_no_dash_sequence_pre_subset)
    subset_no_dash_end = subset_no_dash_start + len(alignment_no_dash_sequence_subset)

    #make mod_no_dash alignment
    mod_subset = mod_no_dash[subset_no_dash_start:subset_no_dash_end]

    return mod_subset





In [38]:
''' 
The input of the function is a dictionary in the format of 'chromosome':[[start,end],[start,end]] 
'''
def region_read_mA_density_calculator (chromosome_coordinates,name,bamfile): 
    data_table = [] 

    #get each chromosome
    for chr_name in chromosome_coordinates:
        for region in chromosome_coordinates[chr_name]:
            region_density = []
            region_base = 0 
            
            region_start_index = int(region[0])
            region_end_index = int(region[1])

            
            for read in bamfile.fetch(chr_name,region_start_index,region_end_index):

                #make an if statement to check a specific read front, middle, end regions 
                #setting read start, end, density, length variables 
                    
                #Get the starting and ending positions of the reads 
                read_start_position = read.reference_start
                read_end_position = read.reference_end
                read_density = 0 
        
                #Get sequence information which shows deletions and insertions 
                sequence = read.get_aligned_pairs(matches_only=False, with_seq = True)

                #make a numpy of the sequence length which eliminates the deletion
                
                read_sequence_insertion_included = ''
                genomic_alignment_sequence_deletion_mistach_included = ''
                
                for item in sequence:
                    if item[0] is None:
                        read_sequence_insertion_included+='-'
                    elif item[1] is None:
                        genomic_alignment_sequence_deletion_mistach_included += '-'
                    else: 
                        read_sequence_insertion_included+=item[2]
                        genomic_alignment_sequence_deletion_mistach_included +=item[2]

                
                read_sequence_insertion_included = read_sequence_insertion_included.upper()
                genomic_alignment_sequence_deletion_mistach_included = genomic_alignment_sequence_deletion_mistach_included.upper()


                genomic_alignment_sequence_deletion_mistach_included_mask = np.array(
                    [char != '-' for char in genomic_alignment_sequence_deletion_mistach_included])

                #take sequence length excluding insertions 
                insertions = read_sequence_insertion_included.count ("-")
                no_insertion_no_deletion_sequence_length = len(read_sequence_insertion_included) 
                
                
                
                #removing reads shorter than 50000 
                #if no_insertion_no_deletion_sequence_length < 50000:
                #    continue 

                #make a mod np array with the length of the read length
                mod=read.modified_bases_forward
                
                #make a mod score with its original length 
                mod_score = np.zeros(len(genomic_alignment_sequence_deletion_mistach_included),)

                
                #make transfer mA positions to mod np array corresponded to their sequence positions 
                try:
                    for indices, values in mod[('A', 0, 'a')]:
                        
                        mod_score[indices] = values
                    mod_score = mod_score[genomic_alignment_sequence_deletion_mistach_included_mask]

                    if read.is_reverse:
                        mod_score = mod_score[::-1]

                # No mod would return KeyError 
                except KeyError:
                    continue
                    


                # if the regions are longer than the reads 
                if (region_end_index - region_start_index) > (read_end_position - read_start_position):
                    # scenario 4: if the reads are inside the region
                    if (region_end_index >= read_end_position) and (region_start_index <= read_start_position): 
                        mod_start = 0
                        mod_end = len(read_sequence_insertion_included)
                    
                    # scenario 5: if the reads cover the later part of the region
                    elif (region_end_index < read_end_position) and (region_start_index > read_start_position): 
                        mod_start = 0
                        mod_end = no_insertion_no_deletion_sequence_length - read_end_position - region_end_index

                    # scenario 6: if the reads cover the starting part of the region 
                    elif (region_end_index > read_end_position) and (region_start_index > read_start_position): 
                        mod_start = region_start_index - read_start_position 
                        mod_end = no_insertion_no_deletion_sequence_length

                        
                
                # if the reads are longer than the region selected 
                elif (region_end_index - region_start_index) <= (read_end_position - read_start_position):
                    # scenario 1: when the defined region is inside the read
                    if (read_start_position <= region_start_index) and (read_end_position >= region_end_index):
                        mod_start = region_start_index - read_start_position 
                        mod_end = region_end_index - read_start_position

                    # scenario 3: when the defined region covers a bit of the end of the read
                    elif (read_end_position < region_end_index) and (read_end_position > region_start_index):
                        mod_start = region_start_index - read_start_position

                        mod_end = no_insertion_no_deletion_sequence_length

                    # scenario 2: when the defined region covers a bit of the beginning of the read
                    elif (read_start_position > region_start_index) and (read_start_position < region_end_index):
                        mod_start = 0
                        mod_end = region_end_index - read_start_position 


                #use the defined starting and ending positons in the region to subset mod numpy
                if (region_start_index - read_start_position) > (no_insertion_no_deletion_sequence_length - insertions):
                    continue
                trimmed_mod_score = mod_subset_producing_step (mod_score,read_sequence_insertion_included,mod_start,mod_end)
                
            
                region_base += (mod_end - mod_start)
                #removing all the zeros 
                mod_no_zeros = trimmed_mod_score[trimmed_mod_score != 0]
                mA = len (mod_no_zeros)
                
                #Getting the total amount of As in the subsetted region of the sequence 
                total_A = read_sequence_insertion_included[mod_start:mod_end].count("A")
                
                
                #calculate read density
                try:
                    read_density = mA / total_A
                    
                except ZeroDivisionError:
                    pass
                region_density.append (read_density)

                
                    
                
            #calculate averaged region density average 
            try:
                region_density_average = sum(region_density)/len(region_density)
            
            except ZeroDivisionError:
                region_density_average = 0
            data_table.append ([chr_name,region,region_density_average,region_base/(region_end_index - region_start_index)])
            #print (chr_name, region_end_index - region_start_index, region_base )

        
    table = tabulate(data_table, headers="firstrow", tablefmt="fancy_grid", floatfmt=".18f")
    
    filename = f"/private/groups/migalab/dan/data_analysis/mC_border_analysis/{name}_region_density_scores.csv"
    with open(filename, "w", newline="") as csvfile:
        writer = csv.writer(csvfile,delimiter="\t")
        writer.writerows(data_table)

    print (name)
    print (table)
    

#region_read_mA_density_calculator(test_dict,'test_dict',bamfile)

#for scenario in scenarios: 
    #region_read_mA_density_calculator (scenario,str(scenario))
    

In [16]:

import unittest
from unittest.mock import MagicMock, patch
class TestRegionReadMADensityCalculator(unittest.TestCase):

    @patch('pysam.AlignmentFile')
    def test_region_read_mA_density_calculator(self, mock_bamfile):
        # Mock chromosome names
        mock_bamfile.references = ['chr1']
        
        # Define mock reads and return values
        mock_reads = []
        for i in range(100):
            mock_read = MagicMock()
            mock_read.reference_start = 100
            mock_read.reference_end = 500
            # Create modified bases with mA every other A position

            mock_read.modified_bases_forward= {('A', 0, 'a'): [(0, 254), (4, 254), (8, 254),(16, 254), (24, 254),(32, 254), (36, 254), (40, 254),(48, 254), (52, 254)]
                                               +[(104, 254), (108, 254), (112, 254),(116, 254), (152, 254)]
                                               +[(j, 250) for j in range(200, 300, 4)]
                                               +[(396, 254)]
                                               }
            
            # Mock the sequence as a long string of 'A's
            mock_read.get_aligned_pairs.return_value = [(j, j, 'ATCG') for j in range(0, 100)]
            
            mock_read.is_reverse = False
            
            mock_reads.append(mock_read)

        # Mock the fetch method
        mock_bamfile.fetch.side_effect = lambda chr_name, start, end: iter(mock_reads) if chr_name == 'chr1' else iter([])

        # Define input
        chromosome_coordinates = {'chr1': [[300, 600]]}
        name = 'test_output'
        
        # Call the function
        region_read_mA_density_calculator(chromosome_coordinates, name, mock_bamfile)
        
        # Check if the file was created and contains the expected data
        with open('test_output_region_density_scores.csv', 'r') as file:
            content = file.read()
        
        # You can add more assertions here to check the file contents
        self.assertIn('chr1', content)
        self.assertIn('[300, 600]', content)

if __name__ == '__main__':
    unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestRegionReadMADensityCalculator))


.
----------------------------------------------------------------------
Ran 1 test in 0.104s

OK


test_output
╒════════╤══════════════╤═════════════════════╤═════════════════════╕
│ chr1   │ [300, 600]   │ 0.520000000000001   │ 66.66666666666667   │
╞════════╪══════════════╪═════════════════════╪═════════════════════╡
╘════════╧══════════════╧═════════════════════╧═════════════════════╛


In [ ]:

import unittest
from unittest.mock import MagicMock, patch
class TestRegionReadMADensityCalculator(unittest.TestCase):

    @patch('pysam.AlignmentFile')
    def test_region_read_mA_density_calculator(self, mock_bamfile):
        # Mock chromosome names
        mock_bamfile.references = ['chr1']
        
        # Define mock reads and return values
        mock_reads = []
        for i in range(1):
            mock_read = MagicMock()
            mock_read.reference_start = 100
            mock_read.reference_end = 500
            # Create modified bases with mA every other A position

            mock_read.modified_bases_forward= {('A', 0, 'a'): 
                                               [(j, 250) for j in range(0, 400, 4)]
                                               
                                               }
            
            # Mock the sequence as a long string of 'A's
            aligned_pairs = []
            n = 0 
            for i in range(200):

                aligned_pairs.append((n, i, 'A'))
                n=n+1
                aligned_pairs.append((n, None, None))
                n+=1
            for i in range (1000):
                aligned_pairs.append((None, i, 'A'))


                

            
            
            mock_read.get_aligned_pairs.return_value = aligned_pairs 
            
            mock_read.is_reverse = False
            
            mock_reads.append(mock_read)

        # Mock the fetch method
        mock_bamfile.fetch.side_effect = lambda chr_name, start, end: iter(mock_reads) if chr_name == 'chr1' else iter([])

        # Define input
        chromosome_coordinates = {'chr1': [[0, 600]]}
        name = 'test_output'
        
        # Call the function
        region_read_mA_density_calculator(chromosome_coordinates, name, mock_bamfile)
        
        # Check if the file was created and contains the expected data
        with open('test_output_region_density_scores.csv', 'r') as file:
            content = file.read()
        
        # You can add more assertions here to check the file contents
        self.assertIn('chr1', content)
        self.assertIn('[0, 600]', content)

if __name__ == '__main__':
    unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestRegionReadMADensityCalculator))


In [ ]:
CDR_dict= check_lists(CDR_dict)
CDR_adjacent = check_lists(CDR_adjacent)
none_CDR_active = check_lists(none_CDR_active)
chromosome_arm_random_region_dict = check_lists(chromosome_arm_random_region_dict)

In [42]:
from joblib import Parallel, delayed

bar_plot_dataset = [CDR_dict, none_CDR_active, chromosome_arm_random_region_dict,CDR_adjacent]
bar_plot_dataset_names = [ 'CENPA_CDR_dict', 'none_CDR_active_dict','chromosome_arm_random_region_dict','CDR_adjacent']
def get_variable_name(var, locals_dict):
    for name, value in locals_dict.items():
        if value is var:
            return name
    return None




for dictionary in range (0, len (bar_plot_dataset)): 

    region_read_mA_density_calculator (bar_plot_dataset[dictionary],bar_plot_dataset_names[dictionary],bamfile)

CENPA_CDR_dict
╒══════════════════╤════════════════════════╤════════════════════════╤═══════════════════════╕
│ chr10_MATERNAL   │ [40290084, 40293094]   │   0.001987377495322512 │     54.08039867109635 │
╞══════════════════╪════════════════════════╪════════════════════════╪═══════════════════════╡
│ chr10_MATERNAL   │ [42094556, 42103101]   │   0.014029956125213433 │ 55.980339379754241236 │
├──────────────────┼────────────────────────┼────────────────────────┼───────────────────────┤
│ chr10_MATERNAL   │ [42137768, 42152903]   │   0.029135198525888143 │ 65.959497852659396244 │
├──────────────────┼────────────────────────┼────────────────────────┼───────────────────────┤
│ chr10_MATERNAL   │ [42213526, 42279389]   │   0.040758446829087656 │ 56.849065484414616378 │
├──────────────────┼────────────────────────┼────────────────────────┼───────────────────────┤
│ chr10_MATERNAL   │ [42279979, 42283132]   │   0.028239643931015607 │ 52.698065334601963627 │
├──────────────────┼───────────────

ValueError: 37713 is not in list